# Make copy of images folder with smaller resized images

### Create mirror folder structure

In [ ]:
#use rsync... modify input and output directory names before running
#include directories with \*/ 
#exclude files with \*
!rsync -a data sml80  --include \*/ --exclude \*

### Define custom dataset loader to save output

Pay attention to required inputs: output directory, image dimensions

In [ ]:
############ USER INPUTS ############
out_dir = 'sml80'
dims = (80, 80)
###

from PIL import Image
from PIL import ImageFile

def pil_loader_saver(path: str) -> Image.Image:
    # To prevent: OSError: image file is truncated (150 bytes not processed)
    # which throws when image truncated
    ImageFile.LOAD_TRUNCATED_IMAGES = True

    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)

        # im_resized = img.resize((64, 64), reducing_gap = 2.0)
        # see PIL documentation for details on optional optimisation: reducing_gap
        # my understanding for speed so could be worse so no optim here
        im_resized = img.resize(dims)

#         im_resized.save('sml/{}'.format(path))
        im_resized.save('{}/{}'.format(out_dir, path))
        
        return im_resized.convert('RGB')

### Define Dataloaders

In [ ]:
#todo Loop through subfolders automatically and process for each

############ USER INPUTS ############
data_dir = 'data/dogImages' #laptop
# data_dir = '/data/dog_images' #workspace
###

import torch
from torchvision import datasets
import torchvision.transforms as transforms 

train_data = datasets.ImageFolder(data_dir + '/train', loader=pil_loader_saver, transform=transforms.ToTensor())
valid_data = datasets.ImageFolder(data_dir + '/valid', loader=pil_loader_saver, transform=transforms.ToTensor())
test_data = datasets.ImageFolder(data_dir + '/test', loader=pil_loader_saver, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=100)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

print("Sample sizes:\nTrain: {}\tValid: {}\tTest: {}".format(len(train_data),len(valid_data),len(test_data)))

### Process by iterating through Dataloader

In [ ]:
processed=0
for batch_idx, (data, target) in enumerate(train_loader):
    processed += len(data)
    print(processed,end='\r')

In [ ]:
processed=0
for batch_idx, (data, target) in enumerate(valid_loader):
    processed += len(data)
    print(processed,end='\r')

In [ ]:
processed=0
for batch_idx, (data, target) in enumerate(test_loader):
    processed += len(data)
    print(processed,end='\r')